In [1]:
import pandas as pd
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

/home/yjtech2/Desktop/yurim/anaconda3/envs/venv-llm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-19 15:47:36.546879: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-19 15:47:36.571783: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-19 15:47:36.943899: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

In [2]:
import json
import torch
from transformers import (
    T5ForConditionalGeneration, 
    T5Tokenizer, 
    AutoModelForTokenClassification, 
    AutoTokenizer
)
from tqdm import tqdm
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

class TextCorrector:
    def __init__(self):
        # 띄어쓰기 모델 초기화
        self.spacing_tokenizer = AutoTokenizer.from_pretrained("fiveflow/roberta-base-spacing")
        self.spacing_model = AutoModelForTokenClassification.from_pretrained("fiveflow/roberta-base-spacing")
        
        # 맞춤법 모델 초기화
        self.spelling_tokenizer = T5Tokenizer.from_pretrained('j5ng/et5-typos-corrector')
        self.spelling_model = T5ForConditionalGeneration.from_pretrained('j5ng/et5-typos-corrector')
        
        # GPU 설정
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.spacing_model = self.spacing_model.to(self.device)
        self.spelling_model = self.spelling_model.to(self.device)
        
        # 띄어쓰기 레이블
        self.spacing_labels = ["UNK", "PAD", "O", "B", "I", "E", "S"]

    # 띄어쓰기 교정
    def correct_spacing(self, text):
        if not isinstance(text, str) or not text.strip():
            return ""
            
        try:
            max_length = 512 - 2
            org_text = text.replace(" ", "")
            results = []
            
            for start_idx in range(0, len(org_text), max_length):
                chunk = org_text[start_idx:start_idx + max_length]
                
                # 토큰화
                tokens = [self.spacing_tokenizer.cls_token_id]
                tokens.extend(sum([self.spacing_tokenizer.encode(char)[1:-1] for char in chunk], []))
                tokens.append(self.spacing_tokenizer.sep_token_id)
                
                # 예측
                with torch.no_grad():
                    inputs = torch.tensor([tokens]).to(self.device)
                    outputs = self.spacing_model(inputs)
                    predictions = torch.argmax(outputs.logits, dim=-1)
                
                # 결과 생성
                result = ""
                for idx, pred in enumerate(predictions.squeeze()[1:-1]):
                    if idx >= len(chunk):
                        break
                    result += chunk[idx]
                    if self.spacing_labels[pred] in ["E", "S"]:
                        result += " "
                        
                results.append(result.strip())
            
            return " ".join(results).strip()
            
        except Exception as e:
            print(f"Spacing error: {str(e)}")
            return text

    # 맞춤법 교정
    def correct_spelling(self, text):
        if not isinstance(text, str) or not text.strip():
            return ""
            
        try:
            max_length = 128 - 2
            chunks = [text[i:i + max_length] for i in range(0, len(text), max_length)]
            results = []
            
            for chunk in chunks:
                inputs = self.spelling_tokenizer(
                    "맞춤법을 고쳐주세요: " + chunk,
                    return_tensors="pt",
                    truncation=True,
                    max_length=max_length
                ).to(self.device)
                
                outputs = self.spelling_model.generate(
                    **inputs,
                    max_length=max_length,
                    num_beams=5,
                    early_stopping=True
                )
                
                result = self.spelling_tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
                if result:
                    results.append(result)
            
            return " ".join(results).strip() or ""
            
        except Exception as e:
            print(f"Spelling error: {str(e)}")
            return ""

    def clean_text(self, text):
        if not isinstance(text, str):
            return ""
        return re.sub('[^\w\s]', '', text)

def load_data(file_path):

    
    df = pd.read_excel(file_path)
    
    print(df.info())
    
    return df

def process_dataset(file_path):
    # 데이터 로드 및 전처리기 초기화
    print('데이터 로드')
    df = load_data(file_path)
    
    corrector = TextCorrector()
    
    # 진행바 설정
    tqdm.pandas(desc="Processing corrections")
    
    # 띄어쓰기 교정
    print("Applying spacing correction...")
    df['spacing_res_sentence'] = df['text'].progress_apply(corrector.correct_spacing)
    
    # 맞춤법 교정
    print("Applying spelling correction...")
    df['spelling_res_sentence'] = df['spacing_res_sentence'].progress_apply(corrector.correct_spelling)
    
    # 텍스트 클리닝
    df['spelling_res_sentence'] = df['spelling_res_sentence'].apply(corrector.clean_text)
    return df

if __name__ == "__main__":
    file_path = '/home/yjtech2/Desktop/yurim/LLM/Data/text_test_data.xlsx'
    result_df = process_dataset(file_path)
    
    print("\n=== Results ===")
    result_df

데이터 로드
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    10 non-null     object
dtypes: object(1)
memory usage: 208.0+ bytes
None
Applying spacing correction...


Processing corrections: 100%|██████████| 10/10 [00:00<00:00, 120.06it/s]


Applying spelling correction...


Processing corrections: 100%|██████████| 10/10 [00:01<00:00,  9.02it/s]


=== Results ===


In [3]:
import json
from hunspell import HunSpell
from tqdm import tqdm
import pandas as pd
import re
from datetime import datetime
from difflib import SequenceMatcher

def hunspell_basic(text):
    try:
        # Hunspell 객체 생성
        hunspell = HunSpell(
            '/home/yjtech2/Desktop/yurim/LLM/Pre_processing/spelling/hunspell/ko_KR.dic',
            '/home/yjtech2/Desktop/yurim/LLM/Pre_processing/spelling/hunspell/ko_KR.aff'
        )

        # 결과 저장용 리스트
        corrected_texts = []

        # 문장에서 단어 추출
        words = re.findall(r'\b\w+\b', text)
        
        # 수정된 문장을 저장할 변수
        corrected_sentence = text
        
        for word in words:
            if not hunspell.spell(word):
                suggestions = hunspell.suggest(word)
                
                if suggestions:
                    corrected_word = suggestions[0]
                    corrected_sentence = re.sub(r'\b' + re.escape(word) + r'\b', corrected_word, corrected_sentence, count=1)
                    # print(f"'{word}'를 '{corrected_word}'로 수정했습니다.")
                else:
                    print(f"'{word}'에 대한 제안이 없습니다.")
        
        corrected_texts.append(corrected_sentence)
        # print(f'\n수정 전 문장: {text}')
        # print(f'\n수정 후 문장: {corrected_sentence}')
        
        return corrected_texts

    except Exception as e:
        print(f"Hunspell 초기화 중 오류 발생: {e}")
        return []

def process_dataframe(df, text_column):
    # 진행바 표시
    tqdm.pandas(desc="Processing spacing correction")
    
    # 맞춤법 교정 적용
    df["res_sentence"] = df[text_column].progress_apply(hunspell_basic)
    
    return df



# 수행
if __name__ == "__main__":

    processed_df = process_dataframe(result_df, "spelling_res_sentence")
    
    def text_clean(text):
        pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
        text = re.sub(pattern, '', text)
        
        pattern = '[^\w\s]'         # 특수기호제거
        text = re.sub(pattern, '', text)

        return text 
    
    processed_df['res_sentence'] = processed_df['res_sentence'].apply(lambda x: x[0] if isinstance(x, list) else x)



Processing spacing correction: 100%|██████████| 10/10 [00:02<00:00,  4.09it/s]


In [7]:
processed_df

,text,spacing_res_sentence,spelling_res_sentence,res_sentence
0,음식물 쓰레기 냄세가 나요,음식물 쓰레기 냄세가 나요,음식물 쓰레기 냄새가 나요,음식물 쓰레기 냄새가 나요
1,전라북도 순창군 ㅇㅇ면 마을 꼴목길에 퇴비를 가져다 놓음\n바람이불면 골목안쪽으로...,전라북도 순창군 ㅇㅇ면 마을 꼴목길에 퇴비를 가져다 놓음 \n바람 이불 면골 목안쪽...,전라북도 순창군 ᄋᄋ 면 마을 꼴 목길에 퇴비를 가져다 놓음 바람 이불 면골 목 안...,전라북도 순차군 아 면 마을 꼴 목기에 퇴비를 가져다 놓음 바람 이불 만골 목 안쪽...
2,인도에 폐기물 컨테이너가 있어 통행 불편 및 악취로 불편합니다.\n발리조치해 주시기...,인도에 폐기물 컨테이너가 있어 통행 불편 및 악취로 불편합니다. \n발 리조치 해주...,인도에 폐기물 컨테이너가 있어 통행 불편 및 냄새로 불편합니다,인도에 폐기물 컨테이너가 있어 통행 불편 및 냄새로 불편합니다
3,이웃집 에서 보일러 기름으로 추정되는 물질이 다량으로 유출되어 주변에 피해를 쥽니다,이웃집에서 보일러 기름으로 추정되는 물질이 다량으로 유출되어 주변에 피해를 쥽니다,이웃집에서 보일러 기름으로 추정되는 물질이 다량으로 유출되어 주변에 피해를 쥽니다,이웃집에서 보일러 기름으로 추정되는 물질이 다량으로 유출되어 주변에 피해를 잡니다
4,동국대 신공학관 근처 하수 처리 냄세 신고,동국대 신공학관 근처 하수 처리 냄세 신고,동국대 신공학관 근처 하수 처리 냄새 신고,동국도 신 공학관 근처 하수 처리 냄새 신고
5,층간소음 문제로 12년된 아파트 하자 점검 서비스가 있나요?\n원룸형 아파트 인데요...,층간 소음 문제로 12년 된 아파트 하자 점검 서비스가 있나요? \n원룸 형아파트인...,층간 소음 문제로 12년 된 아파트 하자 점검 서비스가 있나요 원룸 형 아파트인데요...,증간 소음 문제로 12년 된 아파트 하자 점검 서비스가 있나요 원룸 형 아파트인데요...
6,건물 옆 주차장에 길고양들이 배설물을 싸고 있습니다,건물 옆 주차장에 길 고양들이 배설물을 싸고 있습니다,건물 옆 주차장에 길 고양들이 배설물을 싸고 있습니다,건물 옆 주차장에 길 고양들이 배설물을 싸고 있습니다
7,공공시설/ 길거리 흡연자가 녀무 많습니다,공공시설 / 길거리 흡연자가 녀무 많습니다,공공시설 길거리 흡연자가 너무 많습니다,공공시설 길거리 흡연자가 너무 많습니다
8,벌래와 모기 하수구냄새가 납니다,벌래와 모기 하수구 냄새가 납니다,벌래와 모기 하수구 냄새가 납니다,벌레와 모기 하수구 냄새가 납니다
9,바람이 동서족에서 불고 불쾌한 냄새가 납니다,바람이 동서족에서 불고 불쾌한 냄새가 납니다,바람이 동서족에서 불고 불쾌한 냄새가 납니다,바람이 동서 족에서 불고 불쾌한 냄새가 납니다


In [14]:
print(processed_df.iloc[5][0])
print()

print(processed_df.iloc[5][2])

print()

print(processed_df.iloc[5][3])

층간소음 문제로 12년된 아파트 하자 점검 서비스가 있나요?
원룸형 아파트 인데요. 옆집과 저희집 사이애 위치한 책장과 벽에 균열이 생겼는지 벽간 소음 발생이 심합니다. 

심지어 옆집 냄새도 넘어오는 데요. 하자 점검 서비스가 있을까요?

층간 소음 문제로 12년 된 아파트 하자 점검 서비스가 있나요 원룸 형 아파트인데요 비스가 있을

증간 소음 문제로 12년 된 아파트 하자 점검 서비스가 있나요 원룸 형 아파트인데요 비소가 있을
